# 🦙1️⃣: Convert PDFs into Images

In [1]:
import os
from pdf2image import convert_from_path
# import fitz

pdf_folder = '/home/exouser/llm_rag_workshop/pdfs'
image_folder = '/home/exouser/llm_rag_workshop/images'

if not os.path.exists(image_folder):
    os.makedirs(image_folder)

for filename in os.listdir(pdf_folder):
    print(f'filename: {filename}')
    if filename.endswith('.pdf'):
        file_path = os.path.join(pdf_folder, filename)
        print(f'file_path: {file_path}')
        output_folder = os.path.join(image_folder, os.path.splitext(filename)[0])
        print(f'output_folder: {output_folder}')
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        images = convert_from_path(file_path)
        for i, image in enumerate(images):
            image_filename = f"page_{i+1:03d}.jpeg"
            image_path = os.path.join(output_folder, image_filename)
            image.save(image_path, 'JPEG')
            print(f'image_path: {image_path}')

print('PDFs converted to images!')

filename: Riemann - Insomnia disorder  State of the science and challenges for the future.pdf
file_path: /home/exouser/llm_rag_workshop/pdfs/Riemann - Insomnia disorder  State of the science and challenges for the future.pdf
output_folder: /home/exouser/llm_rag_workshop/images/Riemann - Insomnia disorder  State of the science and challenges for the future
image_path: /home/exouser/llm_rag_workshop/images/Riemann - Insomnia disorder  State of the science and challenges for the future/page_001.jpeg
image_path: /home/exouser/llm_rag_workshop/images/Riemann - Insomnia disorder  State of the science and challenges for the future/page_002.jpeg
image_path: /home/exouser/llm_rag_workshop/images/Riemann - Insomnia disorder  State of the science and challenges for the future/page_003.jpeg
image_path: /home/exouser/llm_rag_workshop/images/Riemann - Insomnia disorder  State of the science and challenges for the future/page_004.jpeg
image_path: /home/exouser/llm_rag_workshop/images/Riemann - Insomn

# 🦙2️⃣: Convert Images into Raw Text

In [2]:
import pytesseract
from PIL import Image

extracted_doc_list = []

for dir, subdirs, files in os.walk(image_folder):
    print(f'dir: {dir}, subdirs: {subdirs}, files: {files}\n')
    
    extracted_doc = {
        'doc_name': '',
        'page': [],
        'extracted_text': []
    }
    
    for file in sorted(files):
        print(f'file: {file}')
        
        if file.endswith('.jpeg'):
            doc_name = dir.split("/")[-1]
            extracted_doc['doc_name'] = doc_name
            page = file.split(".")[0].split("_")[-1]
            extracted_doc['page'].append(page)
            print(f'doc_name: {doc_name}, page: {page}')
            
            file_path = os.path.join(dir, file)
            image = Image.open(file_path)
            extracted_text = pytesseract.image_to_string(image)
            extracted_doc['extracted_text'].append(extracted_text)
            
    if extracted_doc['page']:
        extracted_doc_list.append(extracted_doc)
                                                  
print('conversion complete!')     

dir: /home/exouser/llm_rag_workshop/images, subdirs: ['Riemann - Insomnia disorder  State of the science and challenges for the future', 'Scarpelli - What about dreams  State of the art and open questions', '.ipynb_checkpoints'], files: []

dir: /home/exouser/llm_rag_workshop/images/Riemann - Insomnia disorder  State of the science and challenges for the future, subdirs: ['.ipynb_checkpoints'], files: ['page_002.jpeg', 'page_005.jpeg', 'page_003.jpeg', 'page_011.jpeg', 'page_012.jpeg', 'page_015.jpeg', 'page_001.jpeg', 'page_004.jpeg', 'page_010.jpeg', 'page_007.jpeg', 'page_014.jpeg', 'page_018.jpeg', 'page_009.jpeg', 'page_006.jpeg', 'page_016.jpeg', 'page_008.jpeg', 'page_013.jpeg', 'page_017.jpeg']

file: page_001.jpeg
doc_name: Riemann - Insomnia disorder  State of the science and challenges for the future, page: 001
file: page_002.jpeg
doc_name: Riemann - Insomnia disorder  State of the science and challenges for the future, page: 002
file: page_003.jpeg
doc_name: Riemann - Insom

In [3]:
for text in extracted_doc_list[0]['extracted_text']:
    print(f'{text.replace("\n","➡️\n")}')      

Received: 26 February 2022➡️
➡️
Revised: 23 March 2022➡️
➡️
® Check for updates➡️
➡️
Accepted: 23 March 2022➡️
➡️
 ➡️
➡️
DOI: 10.1111/jsr.13604➡️
➡️
REVIEW ARTICLE➡️
➡️
 ➡️
➡️
Insomnia disorder: State of the science and challenges➡️
➡️
for the future➡️
➡️
Dieter Riemann*?® |➡️
Anna F. Johann? ® |➡️
➡️
Eus J. W. Van Someren® ©➡️
➡️
1Department of Psychiatry and➡️
Psychotherapy, Medical Center - University of➡️
Freiburg, Faculty of Medicine, Freiburg,➡️
Germany➡️
➡️
?Center for Basics in NeuroModulation➡️
(NeuroModulBasics), Faculty of Medicine,➡️
University of Freiburg, Freiburg, Germany➡️
➡️
3sir Jules Thorn Sleep and Circadian➡️
Neuroscience Institute, Nuffield Department➡️
of Clinical Neurosciences, University➡️
➡️
of Oxford, Oxford, UK➡️
➡️
“Big Health Ltd, London, UK➡️
5Big Health Ltd, San Francisco, California, USA➡️
➡️
“Institute of Medical Psychology and Medical➡️
Sociology, Faculty of Medicine, University of➡️
Freiburg, Freiburg, Germany➡️
➡️
7Department of Psychological Method

# 🦙3️⃣🅰️: Parse Raw Text: Join Split Sentences

In [40]:
sentence_num = 0
paragraph_num = 0

structured_texts = {}

sentence_endings = {'.', '?', '!'}

sentence_groups = []

end_doc = -1
end_page = -1

for text in extracted_doc_list[end_doc]['extracted_text'][:end_page]:
    #print(f'{text.replace("\n","➡️\n")}')
    lines = text.split('\n')
    #print(lines)
    current_sentence_group = []
    for line in lines:
        whitespace_stripped_line = line.strip()
        if whitespace_stripped_line != '':
            if whitespace_stripped_line[-1] in sentence_endings:
                # Current line ends properly
                current_sentence_group.append(whitespace_stripped_line)
                sentence_groups.append(' '.join(current_sentence_group))
                
                current_sentence_group = []
            else:
                # Current line does not end properly
                current_sentence_group.append(whitespace_stripped_line)
                
print('split sentences joined!')

split sentences joined!


In [41]:
for sentence_group in sentence_groups:
    print(sentence_group)
    print("⏹️")

® Check for updates Received: 1 March 2022 Revised: 28 March 2022 Accepted: 28 March 2022 DOI: 10.1111/jsr.13609 REVIEW ARTICLE What about dreams? State of the art and open questions Serena Scarpellit® | Valentina Alfonsit® | Maurizio Gorgoni??® | Luigi De Gennaro*? © 1Department of Psychology, Sapienza University of Rome, Rome, Italy Summary Body and Action Lab, IRCCS Fondazione Several studies have tried to identify the neurobiological bases of dream experiences, Santa Lucia, Rome, Ital : 5 ¢ ane HSI SOME ANY, nevertheless some questions are still at the centre of the debate. Here, we summarise Correspondence the main open issues concerning the neuroscientific study of dreaming. After overcom- Serena Scarpelli, Department of Psychology, Sapienza University of Rome, Via dei Marsi 78, 00185, Rome, Italy. have focussed on the specific functional or structural brain features predicting dream Email: serena.scarpelli@uniroma1.it ing the rapid eye movement (REM) - non-REM (NREM) sleep dicho

# 🦙3️⃣🅱️: Parse Raw Text: Split by Sentence Endings

In [38]:
import nltk
nltk.download('punkt')

split_sentences = [nltk.sent_tokenize(x) for x in sentence_groups]
page_sentences = []

for sentence in split_sentences:
    #print(sentence)
    page_sentences += sentence
    
for page_sentence in page_sentences:
    print(f'{page_sentence}')
    print("⏹️")

® Check for updates Received: 1 March 2022 Revised: 28 March 2022 Accepted: 28 March 2022 DOI: 10.1111/jsr.13609 REVIEW ARTICLE What about dreams?
⏹️
State of the art and open questions Serena Scarpellit® | Valentina Alfonsit® | Maurizio Gorgoni?
⏹️
?® | Luigi De Gennaro*?
⏹️
© 1Department of Psychology, Sapienza University of Rome, Rome, Italy Summary Body and Action Lab, IRCCS Fondazione Several studies have tried to identify the neurobiological bases of dream experiences, Santa Lucia, Rome, Ital : 5 ¢ ane HSI SOME ANY, nevertheless some questions are still at the centre of the debate.
⏹️
Here, we summarise Correspondence the main open issues concerning the neuroscientific study of dreaming.
⏹️
After overcom- Serena Scarpelli, Department of Psychology, Sapienza University of Rome, Via dei Marsi 78, 00185, Rome, Italy.
⏹️
have focussed on the specific functional or structural brain features predicting dream Email: serena.scarpelli@uniroma1.it ing the rapid eye movement (REM) - non-REM

[nltk_data] Downloading package punkt to /home/exouser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 🦙4️⃣: Create Embeddings

# 🦙5️⃣: Query Embeddings and Prompt LLM